In [84]:
import numpy as np
import pandas as pd
import collections
import jieba
from keras.preprocessing.sequence import pad_sequences  #序列预处理 序列填充
from keras.utils import to_categorical,plot_model   # 将类别向量转换为二进制（只有0和1）的矩阵类型表示
from keras.models import Sequential   # 序贯模型是函数式模型的简略版，为最简单的线性、从头到尾的结构顺序，不分叉，是多个网络层的线性堆叠。
from keras.layers import Embedding, LSTM, Dense

In [85]:
# train_data_url = "https://worksheets.codalab.org/rest/bundles/0x0161fd2fb40d4dd48541c2643d04b0b8/contents/blob/"
# test_data_url = "https://worksheets.codalab.org/rest/bundles/0x1f96bc12222641209ad057e762910252/contents/blob/"

In [86]:
def get_json_data(path):
    # read data
    data_df = pd.read_json(path)
    # change row and colunm
    data_df = data_df.transpose()
    # change colunm order
    data_df = data_df[['query', 'label']]
    return data_df

In [87]:
train_data_df = get_json_data(path="train.json")
test_data_df = get_json_data(path="dev.json")

train_data_df.head()

,query,label
0,今天东莞天气如何,weather
1,从观音桥到重庆市图书馆怎么走,map
2,鸭蛋怎么腌？,cookbook
3,怎么治疗牛皮癣,health
4,唠什么,chat


In [88]:
test_data_df.head()

,query,label
0,毛泽东的诗哦。,poetry
1,有房有车吗微笑,chat
2,2013年亚洲冠军联赛恒广州恒大比赛时间。,match
3,若相惜不弃下一句是什么？,poetry
4,苹果翻译成英语,translation


In [89]:
train_data_df.describe()

,query,label
count,2299,2299
unique,2299,31
top,翻译慷慨激昂,chat
freq,1,455


In [90]:
test_data_df.describe()

,query,label
count,770,770
unique,770,31
top,帮我链接到新浪网,chat
freq,1,154


In [91]:
# 获取所有标签，也就是分类的类别
# lables = list(set(train_data_df['label'].tolist()))
# 所有标签
labels = ['website', 'tvchannel', 'lottery', 'chat', 'match',
          'datetime', 'weather', 'bus', 'novel', 'video', 'riddle',
          'calc', 'telephone', 'health', 'contacts', 'epg', 'app', 'music',
          'cookbook', 'stock', 'map', 'message', 'poetry', 'cinemas', 'news',
          'flight', 'translation', 'train', 'schedule', 'radio', 'email']

label_numbers = len(labels)
label_numbers

31

In [92]:
# 标签和对应ID的映射字典
label_index_dict = dict([(label, index) for index, label in enumerate(labels)])
index_label_dict = dict([(index, label) for index, label in enumerate(labels)])

In [93]:
# 结巴分词 对元数据进行处理

seg_list = jieba.cut("他来到了网易杭研大厦")  # 默认精确模式
list(seg_list)

['他', '来到', '了', '网易', '杭研', '大厦']

In [94]:
# 序列化

def use_jieba_cut(one_sentence):
    return list(jieba.cut(one_sentence))

train_data_df['cut_query'] = train_data_df['query'].apply(use_jieba_cut)
test_data_df['cut_query'] = test_data_df['query'].apply(use_jieba_cut)

train_data_df.head(10)

,query,label,cut_query
0,今天东莞天气如何,weather,"[今天, 东莞, 天气, 如何]"
1,从观音桥到重庆市图书馆怎么走,map,"[从, 观音桥, 到, 重庆市, 图书馆, 怎么, 走]"
2,鸭蛋怎么腌？,cookbook,"[鸭蛋, 怎么, 腌, ？]"
3,怎么治疗牛皮癣,health,"[怎么, 治疗, 牛皮癣]"
4,唠什么,chat,"[唠, 什么]"
5,阳澄湖大闸蟹的做法。,cookbook,"[阳澄湖, 大闸蟹, 的, 做法, 。]"
6,昆山大润发在哪里,map,"[昆山, 大润发, 在, 哪里]"
7,红烧肉怎么做？嗯？,cookbook,"[红烧肉, 怎么, 做, ？, 嗯, ？]"
8,南京到厦门的火车票,train,"[南京, 到, 厦门, 的, 火车票]"
9,6的平方,calc,"[6, 的, 平方]"


In [95]:
test_data_df.head(10)

,query,label,cut_query
0,毛泽东的诗哦。,poetry,"[毛泽东, 的, 诗, 哦, 。]"
1,有房有车吗微笑,chat,"[有房, 有车, 吗, 微笑]"
2,2013年亚洲冠军联赛恒广州恒大比赛时间。,match,"[2013, 年, 亚洲, 冠军联赛, 恒, 广州, 恒大, 比赛, 时间, 。]"
3,若相惜不弃下一句是什么？,poetry,"[若, 相惜, 不弃, 下, 一句, 是, 什么, ？]"
4,苹果翻译成英语,translation,"[苹果, 翻译成, 英语]"
5,翻译光大银行,translation,"[翻译, 光大银行]"
6,哪天的？,chat,"[哪天, 的, ？]"
7,无锡到阜阳怎么坐汽车？,bus,"[无锡, 到, 阜阳, 怎么, 坐, 汽车, ？]"
8,孜然排骨怎么做？,cookbook,"[孜然, 排骨, 怎么, 做, ？]"
9,娃咋么杨你因为呵呵演完呀不到你,chat,"[娃, 咋, 么, 杨, 你, 因为, 呵呵, 演完, 呀, 不到, 你]"


In [96]:
# 获取数据的所有词汇

def get_all_vocabulary(data, colunm_name):
    train_vocabulary_list = []
    max_cut_query_lenth = 0
    
    for cut_query in data[colunm_name]:
        if len(cut_query) > max_cut_query_lenth:
            max_cut_query_lenth = len(cut_query)
        train_vocabulary_list += cut_query
    return train_vocabulary_list, max_cut_query_lenth

train_vocabulary_list, max_cut_query_lenth = get_all_vocabulary(train_data_df, 'cut_query')
print('Number of words:', len(train_vocabulary_list))

Number of words: 11498


In [97]:
print('Max_cut_query_lenth:', max_cut_query_lenth)

Max_cut_query_lenth: 26


In [98]:
test_vocabulary_list, test_max_cut_query_lenth = get_all_vocabulary(test_data_df, 'cut_query')
print('Test_max_cut_query_lenth:', test_max_cut_query_lenth)

Test_max_cut_query_lenth: 46


In [99]:
train_vocabulary_list[:10]

['今天', '东莞', '天气', '如何', '从', '观音桥', '到', '重庆市', '图书馆', '怎么']

In [100]:
train_vocabulary_counter = collections.Counter(train_vocabulary_list)
print('Number of different words:', len(train_vocabulary_counter.keys()))

Number of different words: 2887


In [101]:
# 不同种类的词汇个数，预留一个位置给不存在的词汇（不存在的词汇标记为0）

max_features = len(train_vocabulary_counter.keys()) + 1
max_features

2888

In [102]:
# 频率最高的10个字
train_vocabulary_counter.most_common(10)

[('的', 605),
 ('。', 341),
 ('我', 320),
 ('你', 297),
 ('怎么', 273),
 ('？', 251),
 ('什么', 210),
 ('到', 165),
 ('给', 154),
 ('做', 148)]

In [103]:
# 统计低频词语
words_times_zero = 0
for word, words_times in train_vocabulary_counter.items():
    if words_times <= 1:
        words_times_zero += 1
print('Word_times_zero:', words_times_zero)
print('Wors_times_zero/all:', words_times_zero / len(train_vocabulary_counter))

Word_times_zero: 1978
Wors_times_zero/all: 0.685140284031867


In [104]:
# 制作词汇字典

def create_train_vocabulary_dict(train_vocabulary_counter):
    word_index, index_word = {}, {}
    index_number = 1
    for word, words_time in train_vocabulary_counter.most_common():
        word_index[word] = index_number
        index_word[index_number] = word
        index_number += 1
    return word_index, index_word

word_index_dict, index_word_dict = create_train_vocabulary_dict(train_vocabulary_counter)

print(word_index_dict['我'], word_index_dict['。'])
        

3 2


In [105]:
print(index_word_dict[55], index_word_dict[1])

播放 的


In [106]:
pq = 0
for index, row in train_data_df.iteritems():
    print(row[0], row[1], row[2])
    pq += 1
    if pq == 10:
        break

今天东莞天气如何 从观音桥到重庆市图书馆怎么走 鸭蛋怎么腌？
weather map cookbook
['今天', '东莞', '天气', '如何'] ['从', '观音桥', '到', '重庆市', '图书馆', '怎么', '走'] ['鸭蛋', '怎么', '腌', '？']


In [107]:
word_index_dict.get('我们', 0)

371

In [108]:
# 向量化数据
def vectorize_data(data, label_index_dict, word_index_dict, max_cut_query_lenth):
    x_train = []
    y_train = []
    for index, row in data.iterrows():
        query_sentence = row[2]
        label = row[1]
        # 字典找不到的情况下用 0 填充
        x = [word_index_dict.get(w, 0) for w in query_sentence]
        y = [label_index_dict[label]]
        x_train.append(x)
        y_train.append(y)
    return (pad_sequences(x_train, maxlen=max_cut_query_lenth), pad_sequences(y_train, maxlen=1))

x_train, y_train = vectorize_data(train_data_df, label_index_dict, word_index_dict, max_cut_query_lenth)
x_test, y_test = vectorize_data(test_data_df, label_index_dict, word_index_dict, test_max_cut_query_lenth)
print(x_train[0], y_train[0])

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0  33 318  27  90] [6]


In [109]:
y_train = to_categorical(y_train, label_numbers)
y_test = to_categorical(y_test, label_numbers)
print(x_train.shape, y_train.shape)

(2299, 26) (2299, 31)


In [ ]:




# 设计模型

def creat_lstm_model(max_features, max_cut_query_lenth, label_numbers):
    model = Sequential()
    model.add(Embedding(input_dim=max_features, output_dim=32, input_length=max_cut_query_lenth))
    model.add(LSTM(units=64, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(label_numbers, activation='softmax'))
    # 尝试使用不同的优化器和不同的优化器配置
    model.compile(loss='categorical_crossentropy',  # categorical_crossentropy：亦称作多类的对数损失，注意使用该目标函数时，需要将标签转化为形如(nb_samples, nb_classes)的二值序列
                  optimizer='adam',
                  metrics=[f1])   # 指标
